# Setup
You will need an image dataset to experiment with, as well as a few Python packages.

# A Dataset to Play With
We will be using the Canadian Institute for Advanced Research image dataset, better known as [CIFAR-10](https://en.wikipedia.org/wiki/CIFAR-10), which consists of 60,000 32x32 pixel color images belonging to different object classes, such as dogs, cats, and airplanes. Relatively, CIFAR is not a very large dataset, but if we were to use the full [TinyImages](https://groups.csail.mit.edu/vision/TinyImages/) dataset, then you would need about 400GB of free disk space, which would probably be a limiting factor.

Credits for the dataset as described in [chapter 3 of this tech report](https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf) go to Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton.

If you’d like to follow along with the code examples in this article, you can [download CIFAR-10 here](https://www.cs.toronto.edu/~kriz/cifar.html), selecting the Python version. You’ll be sacrificing 163MB of disk space:

When you download and unzip the folder, you’ll discover that the files are not human-readable image files. They have actually been serialized and saved in batches using [cPickle](https://docs.python.org/2/library/pickle.html).

While we won’t consider [pickle](https://realpython.com/python-pickle-module/) or cPickle in this article, other than to extract the CIFAR dataset, it’s worth mentioning that the Python pickle module has the key advantage of being able to serialize any Python object without any extra code or transformation on your part. It also has a potentially serious disadvantage of posing a security risk and not coping well when dealing with very large quantities of data.

The following code unpickles each of the five batch files and loads all of the images into a NumPy array:

In [1]:
import numpy as np
import pickle
from pathlib import Path

# Path to the unzipped CIFAR data
data_dir = Path("../../../data/cifar-10/cifar-10-batches-py/")

# Unpickle function provided by the CIFAR hosts
def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict

images, labels = [], []
for batch in data_dir.glob("data_batch_*"):
    batch_data = unpickle(batch)
    for i, flat_im in enumerate(batch_data[b"data"]):
        im_channels = []
        # Each image is flattened, with channels in order of R, G, B
        for j in range(3):
            im_channels.append(
                flat_im[j * 1024 : (j + 1) * 1024].reshape((32, 32))
            )
        # Reconstruct the original image
        images.append(np.dstack((im_channels)))
        # Save the label
        labels.append(batch_data[b"labels"][i])

print("Loaded CIFAR-10 training set:")
print(f" - np.shape(images)     {np.shape(images)}")
print(f" - np.shape(labels)     {np.shape(labels)}")


Loaded CIFAR-10 training set:
 - np.shape(images)     (50000, 32, 32, 3)
 - np.shape(labels)     (50000,)


All the images are now in RAM in the `images` [variable](https://realpython.com/python-variables/), with their corresponding meta data in `labels`, and are ready for you to manipulate. Next, you can install the Python packages you’ll use for the three methods.

**Note:** That last code block used f-strings. You can read more about them in [Python’s F-String for String Interpolation and Formatting](https://realpython.com/python-f-strings/).

### Setup for Storing Images on Disk[](https://realpython.com/storing-images-in-python/#setup-for-storing-images-on-disk "Permanent link")

You’ll need to set up your environment for the default method of saving and accessing these images from disk. This article will assume you have Python 3.x installed on your system, and will use `Pillow` for the image manipulation:

Shell

``` sh
pip install Pillow
```

Alternatively, if you prefer, you can install it using [Anaconda](https://anaconda.org/conda-forge/pillow):

Shell

``` sh
conda install -c conda-forge pillow
```

**Note:** `PIL` is the original version of the Python Imaging Library, which is no longer maintained and is not compatible with Python 3.x. If you have previously installed `PIL`, make sure to uninstall it before installing `Pillow`, as they can’t exist together.

Now you’re ready for storing and reading images from disk.

### Getting Started With LMDB[](https://realpython.com/storing-images-in-python/#getting-started-with-lmdb "Permanent link")

[LMDB](https://en.wikipedia.org/wiki/Lightning_Memory-Mapped_Database), sometimes referred to as the “Lightning Database,” stands for Lightning Memory-Mapped Database because it’s fast and uses memory-mapped files. It’s a key-value store, not a relational database.

In terms of implementation, LMDB is a B+ tree, which basically means that it is a tree-like graph structure stored in memory where each key-value element is a node, and nodes can have many children. Nodes on the same level are linked to one another for fast traversal.

Critically, key components of the B+ tree are set to correspond to the page size of the host operating system, maximizing efficiency when accessing any key-value pair in the database. Since LMDB high-performance heavily relies on this particular point, LMDB efficiency has been shown to be dependent on the underlying file system and its implementation.

Another key reason for the efficiency of LMDB is that it is memory-mapped. This means that **it returns direct pointers to the memory addresses of both keys and values**, without needing to copy anything in memory as most other databases do.

Those who want to dive into a bit more of the internal implementation details of B+ trees can check out [this article on B+ trees](http://www.cburch.com/cs/340/reading/btree/index.html) and then play with [this visualization of node insertion](https://www.cs.usfca.edu/~galles/visualization/BPlusTree.html).

If B+ trees don’t interest you, don’t worry. You don’t need to know much about their internal implementation in order to use LMDB. We will be using the [Python binding](https://realpython.com/python-bindings-overview/) for the LMDB C library, which can be installed via pip:

Shell

```
pip install lmdb
```

You also have the option of installing via [Anaconda](https://anaconda.org/conda-forge/python-lmdb):

Shell

```
conda install -c conda-forge python-lmdb
```

Check that you can `import lmdb` from a Python shell, and you’re good to go.

## Storing a Single Image[](https://realpython.com/storing-images-in-python/#storing-a-single-image "Permanent link")

Now that you have a general overview of the methods, let’s dive straight in and look at a quantitative comparison of the basic tasks we care about: **how long it takes to read and write files, and how much disk memory will be used.** This will also serve as a basic introduction to how the methods work, with code examples of how to use them.

When I refer to “files,” I generally mean a lot of them. However, it is important to make a distinction since some methods may be optimized for different operations and quantities of files.

For the purposes of experimentation, **we can compare the performance between various quantities of files, by factors of 10 from a single image to 100,000 images.** Since our five batches of CIFAR-10 add up to 50,000 images, we can use each image twice to get to 100,000 images.

To prepare for the experiments, you will want to create a folder for each method, which will contain all the database files or images, and save the paths to those directories in variables:

In [2]:
from pathlib import Path

disk_dir = Path("data/disk/")
lmdb_dir = Path("data/lmdb/")
hdf5_dir = Path("data/hdf5/")

`Path` does not automatically create the folders for you unless you specifically ask it to:

In [3]:
disk_dir.mkdir(parents=True, exist_ok=True)
lmdb_dir.mkdir(parents=True, exist_ok=True)
hdf5_dir.mkdir(parents=True, exist_ok=True)

Now you can move on to running the actual experiments, with code examples of how to perform basic tasks with the three different methods. We can use the `timeit` module, which is included in the Python standard library, to help time the experiments.

Although the main purpose of this article is not to learn the APIs of the different Python packages, it is helpful to have an understanding of how they can be implemented. We will go through the general principles alongside all the code used to conduct the storing experiments.

### Storing to Disk[](https://realpython.com/storing-images-in-python/#storing-to-disk "Permanent link")

Our input for this experiment is a single image `image`, currently in memory as a NumPy array. You want to save it first to disk as a `.png` image, and name it using a unique image ID `image_id`. This can be done using the `Pillow` package you installed earlier:

In [4]:
from PIL import Image
import csv

def store_single_disk(image, image_id, label):
    """ Stores a single image as a .png file on disk.
        Parameters:
        ---------------
        image       image array, (32, 32, 3) to be stored
        image_id    integer unique ID for image
        label       image label
    """
    Image.fromarray(image).save(disk_dir / f"{image_id}.png")

    with open(disk_dir / f"{image_id}.csv", "wt") as csvfile:
        writer = csv.writer(
            csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        writer.writerow([label])

This saves the image. In all realistic applications, you also care about the meta data attached to the image, which in our example dataset is the image label. When you’re storing images to disk, there are several options for saving the meta data.

One solution is to encode the labels into the image name. This has the advantage of not requiring any extra files.

However, it also has the big disadvantage of forcing you to deal with all the files whenever you do anything with labels. Storing the labels in a separate file allows you to play around with the labels alone, without having to load the images. Above, I have stored the labels in a separate `.csv` files for this experiment.

Now let’s move on to doing the exact same task with LMDB.

In [5]:

class CIFAR_Image:
    def __init__(self, image, label):
        # Dimensions of image for reconstruction - not really necessary 
        # for this dataset, but some datasets may include images of 
        # varying sizes
        self.channels = image.shape[2]
        self.size = image.shape[:2]

        self.image = image.tobytes()
        self.label = label

    def get_image(self):
        """ Returns the image as a numpy array. """
        image = np.frombuffer(self.image, dtype=np.uint8)
        return image.reshape(*self.size, self.channels)

Secondly, because LMDB is memory-mapped, new databases need to know how much memory they are expected to use up. This is relatively straightforward in our case, but it can be a massive pain in other cases, which you will see in more depth in a later section. LMDB calls this variable the `map_size`.

Finally, read and write operations with LMDB are performed in `transactions`. You can think of them as similar to those of a traditional database, consisting of a group of operations on the database. This may look already significantly more complicated than the disk version, but hang on and keep reading!

With those three points in mind, let’s look at the code to save a single image to a LMDB:

In [6]:

import lmdb
import pickle

def store_single_lmdb(image, image_id, label):
    """ Stores a single image to a LMDB.
        Parameters:
        ---------------
        image       image array, (32, 32, 3) to be stored
        image_id    integer unique ID for image
        label       image label
    """
    map_size = image.nbytes * 10

    # Create a new LMDB environment
    env = lmdb.open(str(lmdb_dir / f"single_lmdb"), map_size=map_size)

    # Start a new write transaction
    with env.begin(write=True) as txn:
        # All key-value pairs need to be strings
        value = CIFAR_Image(image, label)
        key = f"{image_id:08}"
        txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()

**Note:** It’s a good idea to calculate the exact number of bytes each key-value pair will take up.

With a dataset of images of varying size, this will be an approximation, but you can use `sys.getsizeof()` to get a reasonable approximation. Keep in mind that `sys.getsizeof(CIFAR_Image)` will only return the size of a class definition, which is 1056, _not_ the size of an instantiated object.

The function will also not be able to fully calculate nested items, [lists](https://realpython.com/python-lists-tuples/), or objects containing references to other objects.

Alternately, you could use [`pympler`](https://pythonhosted.org/Pympler/#usage-examples) to save you some calculations by determining the exact size of an object.

You are now ready to save an image to LMDB. Lastly, let’s look at the final method, HDF5.

### Storing With HDF5[](https://realpython.com/storing-images-in-python/#storing-with-hdf5 "Permanent link")

Remember that an HDF5 file can contain more than one dataset. In this rather trivial case, you can create two datasets, one for the image, and one for its meta data:

In [8]:

import h5py

def store_single_hdf5(image, image_id, label):
    """ Stores a single image to an HDF5 file.
        Parameters:
        ---------------
        image       image array, (32, 32, 3) to be stored
        image_id    integer unique ID for image
        label       image label
    """
    # Create a new HDF5 file
    file = h5py.File(hdf5_dir / f"{image_id}.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "image", np.shape(image), h5py.h5t.STD_U8BE, data=image
    )
    meta_set = file.create_dataset(
        "meta", np.shape(label), h5py.h5t.STD_U8BE, data=label
    )
    file.close()

`h5py.h5t.STD_U8BE` specifies the type of data that will be stored in the dataset, which in this case is unsigned 8-bit integers. You can see a full [list of HDF’s predefined datatypes here](http://api.h5py.org/h5t.html).

**Note:** The choice of datatype will strongly affect the runtime and storage requirements of HDF5, so it is best to choose your minimum requirements.

Now that we have reviewed the three methods of saving a single image, let’s move on to the next step.

### Experiments for Storing a Single Image[](https://realpython.com/storing-images-in-python/#experiments-for-storing-a-single-image "Permanent link")

Now you can put all three functions for saving a single image into a [dictionary](https://realpython.com/python-dicts/), which can be called later during the timing experiments:

In [9]:
_store_single_funcs = dict(
    disk=store_single_disk, lmdb=store_single_lmdb, hdf5=store_single_hdf5
)

Finally, everything is ready for conducting the timed experiment. Let’s try saving the first image from CIFAR and its corresponding label, and storing it in the three different ways:

In [10]:
from timeit import timeit

store_single_timings = dict()

for method in ("disk", "lmdb", "hdf5"):
    t = timeit(
        "_store_single_funcs[method](image, 0, label)",
        setup="image=images[0]; label=labels[0]",
        number=1,
        globals=globals(),
    )
    store_single_timings[method] = t
    print(f"Method: {method}, Time usage: {t}")

Method: disk, Time usage: 0.0340491000097245
Method: lmdb, Time usage: 0.010862199997063726
Method: hdf5, Time usage: 0.0009256000048480928


**Note:** While you’re playing around with LMDB, you may see a `MapFullError: mdb_txn_commit: MDB_MAP_FULL: Environment mapsize limit reached` error. It’s important to note that LMDB does **not** overwrite preexisting values, even if they have the same key.

This contributes to the fast write time, but it also means that if you store an image more than once in the same LMDB file, then you will use up the map size. If you run a store function, be sure to delete any preexisting LMDB files first.

Remember that we’re interested in runtime, displayed here in seconds, and also the memory usage:

|Method|Save Single Image + Meta|Memory|
|---|---|---|
|Disk|1.915 ms|8 K|
|LMDB|1.203 ms|32 K|
|HDF5|8.243 ms|8 K|

There are two takeaways here:

1. All of the methods are trivially quick.
2. In terms of disk usage, LMDB uses more.

Clearly, despite LMDB having a slight performance lead, we haven’t convinced anyone why to not just store images on disk. After all, it’s a human readable format, and you can open and view them from any file system browser! Well, it’s time to look at a lot more images…

## Storing Many Images[](https://realpython.com/storing-images-in-python/#storing-many-images "Permanent link")

You have seen the code for using the various storage methods to save a single image, so now we need to adjust the code to save many images and then run the timed experiment.



### Adjusting the Code for Many Images[](https://realpython.com/storing-images-in-python/#adjusting-the-code-for-many-images "Permanent link")

Saving _multiple_ images as `.png` files is as straightforward as calling `store_single_method()` multiple times. But this isn’t true for LMDB or HDF5, since you don’t want a different database file for each image. Rather, you want to put all of the images into one or more files.

You will need to slightly alter the code and create three new functions that accept multiple images, `store_many_disk()`, `store_many_lmdb()`, and `store_many_hdf5`:

In [12]:
def store_many_disk(images, labels):
    """ Stores an array of images to disk
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    # Save all the images one by one
    for i, image in enumerate(images):
        Image.fromarray(image).save(disk_dir / f"{i}.png")

    # Save all the labels to the csv file
    with open(disk_dir / f"{num_images}.csv", "w") as csvfile:
        writer = csv.writer(
            csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        for label in labels:
            # This typically would be more than just one value per row
            writer.writerow([label])

def store_many_lmdb(images, labels):
    """ Stores an array of images to LMDB.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    map_size = num_images * images[0].nbytes * 10

    # Create a new LMDB DB for all the images
    env = lmdb.open(str(lmdb_dir / f"{num_images}_lmdb"), map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
        for i in range(num_images):
            # All key-value pairs need to be Strings
            value = CIFAR_Image(images[i], labels[i])
            key = f"{i:08}"
            txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()

def store_many_hdf5(images, labels):
    """ Stores an array of images to HDF5.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    # Create a new HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_many.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "images", np.shape(images), h5py.h5t.STD_U8BE, data=images
    )
    meta_set = file.create_dataset(
        "meta", np.shape(labels), h5py.h5t.STD_U8BE, data=labels
    )
    file.close()

So you could store more than one file to disk, the image files method was altered to loop over each image in the list. For LMDB, a loop is also needed since we are creating a `CIFAR_Image` object for each image and its meta data.

The smallest adjustment is with the HDF5 method. In fact, there’s hardly an adjustment at all! HFD5 files have no limitation on file size aside from external restrictions or dataset size, so all the images were stuffed into a single dataset, just like before.

Next, you will need to prepare the dataset for the experiments by increasing its size.

### Preparing the Dataset[](https://realpython.com/storing-images-in-python/#preparing-the-dataset "Permanent link")

Before running the experiments again, let’s first double our dataset size so that we can test with up to 100,000 images:

In [13]:

cutoffs = [10, 100, 1000, 10000, 100000]

# Let's double our images so that we have 100,000
images = np.concatenate((images, images), axis=0)
labels = np.concatenate((labels, labels), axis=0)

# Make sure you actually have 100,000 images and labels
print(np.shape(images))
print(np.shape(labels))

(100000, 32, 32, 3)
(100000,)


Now that there are enough images, it’s time for the experiment.

### Experiment for Storing Many Images[](https://realpython.com/storing-images-in-python/#experiment-for-storing-many-images "Permanent link")

As you did with reading many images, you can create a dictionary handling all the functions with `store_many_` and run the experiments:

In [14]:
_store_many_funcs = dict(
    disk=store_many_disk, lmdb=store_many_lmdb, hdf5=store_many_hdf5
)

from timeit import timeit

store_many_timings = {"disk": [], "lmdb": [], "hdf5": []}

for cutoff in cutoffs:
    for method in ("disk", "lmdb", "hdf5"):
        t = timeit(
            "_store_many_funcs[method](images_, labels_)",
            setup="images_=images[:cutoff]; labels_=labels[:cutoff]",
            number=1,
            globals=globals(),
        )
        store_many_timings[method].append(t)

        # Print out the method, cutoff, and elapsed time
        print(f"Method: {method}, Time usage: {t}")

Method: disk, Time usage: 0.005365499993786216
Method: lmdb, Time usage: 0.005308400010108016
Method: hdf5, Time usage: 0.0009045000042533502
Method: disk, Time usage: 0.032640499994158745
Method: lmdb, Time usage: 0.006935099998372607
Method: hdf5, Time usage: 0.0009580999903846532
Method: disk, Time usage: 0.31937180100067053
Method: lmdb, Time usage: 0.024016100011067465
Method: hdf5, Time usage: 0.002382300008321181
Method: disk, Time usage: 3.2408651019941317
Method: lmdb, Time usage: 0.1679007999919122
Method: hdf5, Time usage: 0.018682599999010563
Method: disk, Time usage: 32.49853784698644
Method: lmdb, Time usage: 1.9063706029992318
Method: hdf5, Time usage: 0.16321090000565164



If you’re following along and running the code yourself, you’ll need to sit back a moment in suspense and wait for 111,110 images to be stored three times each to your disk, in three different formats. You’ll also need to say goodbye to approximately 2 GB of disk space.

Now for the moment of truth! **How long did all of that storing take?** A picture is worth a thousand words:

[![store-many](https://files.realpython.com/media/store_many.273573157770.png)](https://files.realpython.com/media/store_many.273573157770.png)

[![store-many-log](https://files.realpython.com/media/store_many_log.29e8ae980ab6.png)](https://files.realpython.com/media/store_many_log.29e8ae980ab6.png)

The first graph shows the normal, unadjusted storage time, highlighting the drastic difference between storing to `.png` files and LMDB or HDF5.

The second graph shows the `log` of the timings, highlighting that HDF5 starts out slower than LMDB but, with larger quantities of images, comes out slightly ahead.

While exact results may vary depending on your machine, **this is why LMDB and HDF5 are worth thinking about.** Here’s the code that generated the above graph:

In [ ]:

import matplotlib.pyplot as plt

def plot_with_legend(
    x_range, y_data, legend_labels, x_label, y_label, title, log=False
):
    """ Displays a single plot with multiple datasets and matching legends.
        Parameters:
        --------------
        x_range         list of lists containing x data
        y_data          list of lists containing y values
        legend_labels   list of string legend labels
        x_label         x axis label
        y_label         y axis label
    """
    plt.style.use("seaborn-whitegrid")
    plt.figure(figsize=(10, 7))

    if len(y_data) != len(legend_labels):
        raise TypeError(
            "Error: number of data sets does not match number of labels."
        )

    all_plots = []
    for data, label in zip(y_data, legend_labels):
        if log:
            temp, = plt.loglog(x_range, data, label=label)
        else:
            temp, = plt.plot(x_range, data, label=label)
        all_plots.append(temp)

    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend(handles=all_plots)
    plt.show()

# Getting the store timings data to display
disk_x = store_many_timings["disk"]
lmdb_x = store_many_timings["lmdb"]
hdf5_x = store_many_timings["hdf5"]

plot_with_legend(
    cutoffs,
    [disk_x, lmdb_x, hdf5_x],
    ["PNG files", "LMDB", "HDF5"],
    "Number of images",
    "Seconds to store",
    "Storage time",
    log=False,
)

plot_with_legend(
    cutoffs,
    [disk_x, lmdb_x, hdf5_x],
    ["PNG files", "LMDB", "HDF5"],
    "Number of images",
    "Seconds to store",
    "Log storage time",
    log=True,
)

Now let’s go on to reading the images back out.

## Reading a Single Image[](https://realpython.com/storing-images-in-python/#reading-a-single-image "Permanent link")

First, let’s consider the case for reading a single image back into an array for each of the three methods.

### Reading From Disk[](https://realpython.com/storing-images-in-python/#reading-from-disk "Permanent link")

Of the three methods, LMDB requires the most legwork when reading image files back out of memory, because of the serialization step. Let’s walk through these functions that read a single image out for each of the three storage formats.

First, read a single image and its meta from a `.png` and `.csv` file:

In [ ]:
def read_single_disk(image_id):
    """ Stores a single image to disk.
        Parameters:
        ---------------
        image_id    integer unique ID for image

        Returns:
        ----------
        image       image array, (32, 32, 3) to be stored
        label       associated meta data, int label
    """
    image = np.array(Image.open(disk_dir / f"{image_id}.png"))

    with open(disk_dir / f"{image_id}.csv", "r") as csvfile:
        reader = csv.reader(
            csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        label = int(next(reader)[0])

    return image, label
    

### Reading From LMDB[](https://realpython.com/storing-images-in-python/#reading-from-lmdb "Permanent link")

Next, read the same image and meta from an LMDB by opening the environment and starting a read transaction:

In [ ]:

def read_single_lmdb(image_id):
    """ Stores a single image to LMDB.
        Parameters:
        ---------------
        image_id    integer unique ID for image

        Returns:
        ----------
        image       image array, (32, 32, 3) to be stored
        label       associated meta data, int label
    """
    # Open the LMDB environment
    env = lmdb.open(str(lmdb_dir / f"single_lmdb"), readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Encode the key the same way as we stored it
        data = txn.get(f"{image_id:08}".encode("ascii"))
        # Remember it's a CIFAR_Image object that is loaded
        cifar_image = pickle.loads(data)
        # Retrieve the relevant bits
        image = cifar_image.get_image()
        label = cifar_image.label
    env.close()

    return image, label

Here are a couple points to not about the code snippet above:

- **Line 13:** The `readonly=True` flag specifies that no writes will be allowed on the LMDB file until the transaction is finished. In database lingo, it’s equivalent to taking a read lock.
- **Line 20:** To retrieve the CIFAR_Image object, you need to reverse the steps we took to pickle it when we were writing it. This is where the `get_image()` of the object is helpful.

This wraps up reading the image back out from LMDB. Finally, you will want to do the same with HDF5.

### Reading From HDF5[](https://realpython.com/storing-images-in-python/#reading-from-hdf5 "Permanent link")

Reading from HDF5 looks very similar to the writing process. Here is the code to open and read the HDF5 file and parse the same image and meta:

In [ ]:
def read_single_hdf5(image_id):
    """ Stores a single image to HDF5.
        Parameters:
        ---------------
        image_id    integer unique ID for image

        Returns:
        ----------
        image       image array, (32, 32, 3) to be stored
        label       associated meta data, int label
    """
    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{image_id}.h5", "r+")

    image = np.array(file["/image"]).astype("uint8")
    label = int(np.array(file["/meta"]).astype("uint8"))

    return image, label

Note that you access the various datasets in the file by indexing the `file` object using the dataset name preceded by a forward slash `/`. As before, you can create a dictionary containing all the read functions:

In [ ]:
_read_single_funcs = dict(
    disk=read_single_disk, lmdb=read_single_lmdb, hdf5=read_single_hdf5
)

With this dictionary prepared, you are ready for running the experiment.

### Experiment for Reading a Single Image[](https://realpython.com/storing-images-in-python/#experiment-for-reading-a-single-image "Permanent link")

You might expect that the experiment for reading a single image in will have somewhat trivial results, but here’s the experiment code:

In [ ]:
from timeit import timeit

read_single_timings = dict()

for method in ("disk", "lmdb", "hdf5"):
    t = timeit(
        "_read_single_funcs[method](0)",
        setup="image=images[0]; label=labels[0]",
        number=1,
        globals=globals(),
    )
    read_single_timings[method] = t
    print(f"Method: {method}, Time usage: {t}")

Here are the results of the experiment for reading a single image:

|Method|Read Single Image + Meta|
|---|---|
|Disk|1.61970 ms|
|LMDB|4.52063 ms|
|HDF5|1.98036 ms|

It’s slightly faster to read the `.png` and `.csv` files directly from disk, but all three methods perform trivially quickly. The experiments we’ll do next are much more interesting.

## Reading Many Images[](https://realpython.com/storing-images-in-python/#reading-many-images "Permanent link")

Now you can adjust the code to read many images at once. This is likely the action you’ll be performing most often, so the runtime performance is essential.

[Remove ads](https://realpython.com/account/join/)

### Adjusting the Code for Many Images[](https://realpython.com/storing-images-in-python/#adjusting-the-code-for-many-images_1 "Permanent link")

Extending the functions above, you can create functions with `read_many_`, which can be used for the next experiments. Like before, it is interesting to compare performance when reading different quantities of images, which are repeated in the code below for reference:

In [ ]:
def read_many_disk(num_images):
    """ Reads image from disk.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Loop over all IDs and read each image in one by one
    for image_id in range(num_images):
        images.append(np.array(Image.open(disk_dir / f"{image_id}.png")))

    with open(disk_dir / f"{num_images}.csv", "r") as csvfile:
        reader = csv.reader(
            csvfile, delimiter=" ", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        for row in reader:
            labels.append(int(row[0]))
    return images, labels

def read_many_lmdb(num_images):
    """ Reads image from LMDB.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []
    env = lmdb.open(str(lmdb_dir / f"{num_images}_lmdb"), readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            cifar_image = pickle.loads(data)
            # Retrieve the relevant bits
            images.append(cifar_image.get_image())
            labels.append(cifar_image.label)
    env.close()
    return images, labels

def read_many_hdf5(num_images):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_many.h5", "r+")

    images = np.array(file["/images"]).astype("uint8")
    labels = np.array(file["/meta"]).astype("uint8")

    return images, labels

_read_many_funcs = dict(
    disk=read_many_disk, lmdb=read_many_lmdb, hdf5=read_many_hdf5
)

With the reading functions stored in a dictionary as with the writing functions, you’re all set for the experiment.

### Experiment for Reading Many Images[](https://realpython.com/storing-images-in-python/#experiment-for-reading-many-images "Permanent link")

You can now run the experiment for reading many images out:

In [ ]:
from timeit import timeit

read_many_timings = {"disk": [], "lmdb": [], "hdf5": []}

for cutoff in cutoffs:
    for method in ("disk", "lmdb", "hdf5"):
        t = timeit(
            "_read_many_funcs[method](num_images)",
            setup="num_images=cutoff",
            number=1,
            globals=globals(),
        )
        read_many_timings[method].append(t)

        # Print out the method, cutoff, and elapsed time
        print(f"Method: {method}, No. images: {cutoff}, Time usage: {t}")

As we did previously, you can graph the read experiment results:

[![read-many-image](https://files.realpython.com/media/read_many.9c4a6dc5bdc0.png)](https://files.realpython.com/media/read_many.9c4a6dc5bdc0.png)

[![read-many-log](https://files.realpython.com/media/read_many_log.594dac8746ad.png)](https://files.realpython.com/media/read_many_log.594dac8746ad.png)

The top graph shows the normal, unadjusted read times, showing the drastic difference between reading from `.png` files and LMDB or HDF5.

In contrast, the graph on the bottom shows the `log` of the timings, highlighting the relative differences with fewer images. Namely, we can see how HDF5 starts out behind but, with more images, becomes consistently faster than LMDB by a small margin.

Plot the Read TimingsShow/Hide

**In practice, the write time is often less critical than the read time.** Imagine that you are training a deep neural network on images, and only half of your entire image dataset fits into RAM at once. Each epoch of training a network requires the entire dataset, and the model needs a few hundred epochs to converge. You will essentially be reading half of the dataset into memory every epoch.

There are several tricks people do, such as training pseudo-epochs to make this slightly better, but you get the idea.

Now, look again at the read graph above. The difference between a 40-second and 4-second read time suddenly is the difference between waiting six hours for your model to train, or forty minutes!

If we view the read and write times on the same chart, we have the following:

[![read-write](https://files.realpython.com/media/read_write.a4f87d39489d.png)](https://files.realpython.com/media/read_write.a4f87d39489d.png)

Plot the Read and Write TimingsShow/Hide

When you’re storing images as `.png` files, there is a big difference between write and read times. However, with LMDB and HDF5, the difference is much less marked. Overall, even if read time is more critical than write time, there is a strong argument for storing images using LMDB or HDF5.

Now that you’ve seen the performance benefits of LMDB and HDF5, let’s look at another crucial metric: disk usage.

